In [1]:
from datetime import datetime
import time

def check_sleep(amount):
    start = datetime.now()
    time.sleep(amount)
    end = datetime.now()
    delta = end-start
    return delta.seconds + delta.microseconds/1000000.

error = sum(abs(check_sleep(0.1)-0.1) for i in range(100))*10
print("Average error is %0.2fms" % error)

Average error is 3.11ms


In [2]:
print(datetime.fromtimestamp(1605498402.136237))

2020-11-15 22:46:42.136237


In [3]:
# import autoreload
# ?autoreload

%load_ext autoreload
%autoreload 2
import torch
from transformers import AutoModel
from transformers import AutoConfig

In [4]:
def aten_mul(node):
    os = node.outputs[0].shape
    return math.prod(os)

In [ ]:
globals()

In [5]:
# config = AutoConfig.from_pretrained("prajjwal1/bert-tiny")
config = AutoConfig.from_pretrained("albert-base-v2")
config.hidden_act = 'gelu_fast'
config.torchscript = True
model = AutoModel.from_config(config)
inputs = torch.randint(1000, size=(1, 100)).long()
# model()
# with torch.onnx.select_model_mode_for_export(model, False):
  # trace, _ = torch.jit._get_trace_graph(model, args=(inputs,))
#     trace = torch.jit.trace(model, (inputs, ))
mo=model.eval()

In [6]:
import pdir

In [7]:
pdir(mo)

property:
    T_destination, _backward_hooks, _buffers, _forward_hooks, _forward_pre_hooks, _load_state_dict_pre_hooks, _modules, _non_persistent_buffers_set, _parameters, _state_dict_hooks, _version, authorized_missing_keys, authorized_unexpected_keys, base_model_prefix, config, dump_patches, embeddings, encoder, keys_to_never_save, name_or_path, pooler, pooler_activation, training
special attribute:
    __annotations__, __class__, __dict__, __doc__, __module__, __weakref__
abstract class:
    __subclasshook__
object customization:
    __format__, __hash__, __init__, __new__, __repr__, __sizeof__, __str__
rich comparison:
    __eq__, __ge__, __gt__, __le__, __lt__, __ne__
attribute access:
    __delattr__, __dir__, __getattr__, __getattribute__, __setattr__
class customization:
    __init_subclass__
pickle:
    __reduce__, __reduce_ex__, __setstate__
descriptor:
    base_model: @property with getter, :obj:`torch.nn.Module`: The main body of the model.
    device: @property with getter

In [8]:
mo

AlbertModel(
  (embeddings): AlbertEmbeddings(
    (word_embeddings): Embedding(30000, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0, inplace=False)
  )
  (encoder): AlbertTransformer(
    (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
    (albert_layer_groups): ModuleList(
      (0): AlbertLayerGroup(
        (albert_layers): ModuleList(
          (0): AlbertLayer(
            (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (attention): AlbertAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (attention_dropout): Dropout(p=0, inplace=False)
      

In [15]:
a=['aten::Int', 'aten::ScalarImplicit', 'aten::abs', 'aten::add', 'aten::add_', 'aten::addmm', 'aten::arange', 'aten::bmm', 'aten::cat', 'aten::clone', 'aten::constant_pad_nd', 'aten::contiguous', 'aten::copy_', 'aten::cumsum', 'aten::div', 'aten::dropout', 'aten::einsum', 'aten::embedding', 'aten::eq', 'aten::expand_as', 'aten::fill_', 'aten::floor_divide', 'aten::full_like', 'aten::gather', 'aten::gelu', 'aten::index', 'aten::layer_norm', 'aten::le', 'aten::log', 'aten::lt', 'aten::masked_fill', 'aten::masked_fill_', 'aten::matmul', 'aten::max', 'aten::mean', 'aten::min', 'aten::mul', 'aten::mul_', 'aten::ne', 'aten::neg', 'aten::ones', 'aten::permute', 'aten::pow', 'aten::relu', 'aten::repeat', 'aten::reshape', 'aten::rsub', 'aten::select', 'aten::size', 'aten::slice', 'aten::softmax', 'aten::split', 'aten::sqrt', 'aten::squeeze', 'aten::stack', 'aten::sub', 'aten::sum', 'aten::t', 'aten::tanh', 'aten::to', 'aten::transpose', 'aten::triu', 'aten::type_as', 'aten::unsqueeze', 'aten::view', 'aten::where', 'aten::zeros', 'aten::zeros_like', 'prim::Constant', 'prim::GetAttr', 'prim::ListConstruct', 'prim::ListUnpack', 'prim::NumToTensor', 'prim::TupleConstruct', 'prim::TupleUnpack']

In [17]:
for ai in a:
    print(f"'{ai}': '{ai.replace('::', '_')}',")

'aten::Int': 'aten_Int',
'aten::ScalarImplicit': 'aten_ScalarImplicit',
'aten::abs': 'aten_abs',
'aten::add': 'aten_add',
'aten::add_': 'aten_add_',
'aten::addmm': 'aten_addmm',
'aten::arange': 'aten_arange',
'aten::bmm': 'aten_bmm',
'aten::cat': 'aten_cat',
'aten::clone': 'aten_clone',
'aten::constant_pad_nd': 'aten_constant_pad_nd',
'aten::contiguous': 'aten_contiguous',
'aten::copy_': 'aten_copy_',
'aten::cumsum': 'aten_cumsum',
'aten::div': 'aten_div',
'aten::dropout': 'aten_dropout',
'aten::einsum': 'aten_einsum',
'aten::embedding': 'aten_embedding',
'aten::eq': 'aten_eq',
'aten::expand_as': 'aten_expand_as',
'aten::fill_': 'aten_fill_',
'aten::floor_divide': 'aten_floor_divide',
'aten::full_like': 'aten_full_like',
'aten::gather': 'aten_gather',
'aten::gelu': 'aten_gelu',
'aten::index': 'aten_index',
'aten::layer_norm': 'aten_layer_norm',
'aten::le': 'aten_le',
'aten::log': 'aten_log',
'aten::lt': 'aten_lt',
'aten::masked_fill': 'aten_masked_fill',
'aten::masked_fill_': 'aten_mas

In [26]:
a = torch.zeros(100, dtype=torch.long)

In [27]:
import sys
sys.getsizeof(a)

64

In [9]:
import time

start_times = dict()
end_times = dict()

def log_start_builder(name):
    def log_start(module, m_in):
        start_times[f'{name}:{module.__class__.__name__}'] = time.perf_counter()
    return log_start
def log_end_builder(name):
    def log_end(module, m_in, m_out):
        end_times[f'{name}:{module.__class__.__name__}'] = time.perf_counter()
#         print(name, module.__class__.__name__, 'end', time.perf_counter())
    return log_end

In [10]:
for name, module in mo.named_modules():
    print(name, module.__class__.__name__)
    module.register_forward_pre_hook(log_start_builder(name))
    module.register_forward_hook(log_end_builder(name))

 AlbertModel
embeddings AlbertEmbeddings
embeddings.word_embeddings Embedding
embeddings.position_embeddings Embedding
embeddings.token_type_embeddings Embedding
embeddings.LayerNorm LayerNorm
embeddings.dropout Dropout
encoder AlbertTransformer
encoder.embedding_hidden_mapping_in Linear
encoder.albert_layer_groups ModuleList
encoder.albert_layer_groups.0 AlbertLayerGroup
encoder.albert_layer_groups.0.albert_layers ModuleList
encoder.albert_layer_groups.0.albert_layers.0 AlbertLayer
encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm LayerNorm
encoder.albert_layer_groups.0.albert_layers.0.attention AlbertAttention
encoder.albert_layer_groups.0.albert_layers.0.attention.query Linear
encoder.albert_layer_groups.0.albert_layers.0.attention.key Linear
encoder.albert_layer_groups.0.albert_layers.0.attention.value Linear
encoder.albert_layer_groups.0.albert_layers.0.attention.attention_dropout Dropout
encoder.albert_layer_groups.0.albert_layers.0.attention.output_dropout Drop

In [11]:
trace = torch.jit.trace(model, inputs) # 1605233162.4006221

/Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/transformers/modeling_albert.py:229: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  position_ids = self.position_ids[:, :seq_length]
/Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/transformers/modeling_utils.py:1673: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  input_tensor.shape == tensor_shape for input_tensor in input_tensors


In [12]:
for k, start in start_times.items():
    print(f'{k}, {(end_times[k]-start)*1000:.3f} ms, {start}, {end_times[k]}')

:AlbertModel, 485.596 ms, 98.592858562, 99.078454732
embeddings:AlbertEmbeddings, 0.386 ms, 98.593064335, 98.593450238
embeddings.word_embeddings:Embedding, 0.058 ms, 98.593096499, 98.593154752
embeddings.position_embeddings:Embedding, 0.037 ms, 98.593165679, 98.593202809
embeddings.token_type_embeddings:Embedding, 0.032 ms, 98.593211549, 98.593243077
embeddings.LayerNorm:LayerNorm, 0.078 ms, 98.5933244, 98.593402786
embeddings.dropout:Dropout, 0.023 ms, 98.593415764, 98.593438446
encoder:AlbertTransformer, 484.573 ms, 98.593459438, 99.078032653
encoder.embedding_hidden_mapping_in:Linear, 0.682 ms, 98.593469199, 98.594151621
encoder.albert_layer_groups.0:AlbertLayerGroup, 39.481 ms, 99.03853302, 99.078014506
encoder.albert_layer_groups.0.albert_layers.0:AlbertLayer, 39.456 ms, 99.038551111, 99.07800696
encoder.albert_layer_groups.0.albert_layers.0.attention:AlbertAttention, 14.133 ms, 99.038580346, 99.052713264
encoder.albert_layer_groups.0.albert_layers.0.attention.query:Linear, 2.921

In [13]:
graph = trace.inlined_graph

In [14]:
graph

graph(%self.1 : __torch__.transformers.modeling_albert.AlbertModel,
      %input_ids : Long(1:100, 100:1, requires_grad=0, device=cpu)):
  %2 : __torch__.torch.nn.modules.activation.Tanh = prim::GetAttr[name="pooler_activation"](%self.1)
  %3 : __torch__.torch.nn.modules.linear.___torch_mangle_14.Linear = prim::GetAttr[name="pooler"](%self.1)
  %4 : __torch__.transformers.modeling_albert.AlbertTransformer = prim::GetAttr[name="encoder"](%self.1)
  %5 : __torch__.transformers.modeling_albert.AlbertEmbeddings = prim::GetAttr[name="embeddings"](%self.1)
  %6 : int = prim::Constant[value=0]() # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/transformers/modeling_albert.py:662:0
  %7 : int = aten::size(%input_ids, %6) # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/transformers/modeling_albert.py:662:0
  %8 : Long(device=cpu) = prim::NumToTensor(%7)
  %9 : int = aten::Int(%8)
  %10 : int = aten::Int(%8)
  %11 : int = prim::Constant[value=1]() # /Users/qqca

In [ ]:
ni=list(graph.nodes())
ni

In [ ]:
ni[0].output().node()

In [ ]:
ni[0].output()

In [ ]:
ni[72].output() # build all input leaf data nodes, then construct the opnode with input and output nodes

In [ ]:
ni[0]

In [ ]:
list(ni[0].outputs())

In [ ]:
list(ni[0].inputs())

In [ ]:
pn=ni[0].input().node()
pn

In [ ]:
gi=list(graph.inputs())
gi

In [ ]:
list(gi[0].node().outputs())[0].type()

In [ ]:
list(gi[1].node().outputs())[1].type().sizes()

In [ ]:
'encoder/encoder.layer.0/encoder.layer.0.attention/encoder.layer.0.attention.output'.split('/')[-1]

In [ ]:
print(pn)
print(gi[1].node())
pn==gi[0].node()

In [ ]:
[i.user for i in gi[0].uses()]

In [ ]:
go=list(graph.outputs())
go

In [ ]:
from graphviz import Digraph

g = Digraph('G')

c0 = Digraph('cluster_0')
c0.body.append('style=filled')
c0.body.append('color=lightgrey')
c0.node_attr.update(style='filled', color='white')
c0.edges([('a0', 'a1'), ('a1', 'a2'), ('a2', 'a3')])
c0.body.append('label ="process #1"')

c2 = Digraph('cluster_0')
c2.body.append('style=filled')
c2.body.append('color=lightgrey')
c2.node_attr.update(style='filled', color='white')
c2.edges([('c0', 'c1'), ('c1', 'c2'), ('c2', 'c3')])
c2.body.append('label ="process #3"')
c2.body.append('color=lightblue')
c0.subgraph(c2)

c1 = Digraph('cluster_1')
c1.node_attr.update(style='filled')
c1.edges([('b0', 'b1'), ('b1', 'b2'), ('b2', 'b3')])
c1.body.append('label ="process #2"')
c1.body.append('color=blue')

g.subgraph(c0)
g.subgraph(c1)

g.edge('start', 'a0')
g.edge('start', 'b0')
g.edge('a1', 'b3')
g.edge('b2', 'a3')
g.edge('a3', 'a0')
g.edge('a3', 'end')
g.edge('b3', 'end')

g.node('start', shape='Mdiamond')
g.node('end', shape='Msquare')

g

In [ ]:
# sub_scopes='encoder.layer.0.attention.output.LayerNorm'.split('.')

In [ ]:
'true' if '' and len(''.split('.')) else 'false'

In [ ]:
sub_scopes='encoder.layer.0.attention.output.LayerNorm'.split('.')
for si in range(len(sub_scopes), 0, -1):
    p_scope = '.'.join(sub_scopes[0:si-1])
    child_scope = '.'.join(sub_scopes[0:si])
    print(si, p_scope, child_scope)